In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import os
import glob
from pathlib import Path
from PIL import Image
import cv2

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D,MaxPooling2D, BatchNormalization, UpSampling2D, Input, ZeroPadding2D, Cropping2D
from keras.preprocessing.image import load_img, array_to_img, img_to_array

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.


In [ ]:
# Define paths in the fancy way, after all we have pathlib now. No more os.path.join...whatever!!
input_dir  = Path('/kaggle/input/denoising-dirty-documents')
train = input_dir / 'train.zip'
train_cleaned = input_dir / 'train_cleaned.zip'
test = input_dir / 'test.zip'


In [ ]:
# importing required modules 
from zipfile import ZipFile 

def extract_zip_file(file_names):
    # opening the zip file in READ mode 
    for file in file_names:
        with ZipFile(file, 'r') as zip: 
            # printing all the contents of the zip file 
            # zip.printdir() 

            # extracting all the files 
            # print('Extracting all the files now...') 
            zip.extractall() 
            # print('Done!')

# The train directory comtaims png files. Let's get all the files and check a few samples
extract_zip_file([train, train_cleaned, test])


In [ ]:
train_images = sorted(glob.glob('train/*.png'))
train_cleaned_images = sorted(glob.glob('train_cleaned/*.png'))
test_images = sorted(glob.glob('test/*.png'))


In [ ]:
def convert_imgs_to_array(images_folder, test=False):
    images = []
    
    for img_dir in images_folder:
        image = load_img(img_dir, color_mode='grayscale', target_size=(258,540,1))
        image = img_to_array(image).astype('float32') / 255.0
        images.append(image)
        
    return np.asarray(images)


In [ ]:
X_train_full = convert_imgs_to_array(train_images)
Y_train_full = convert_imgs_to_array(train_cleaned_images)
X_test = convert_imgs_to_array(test_images)


In [ ]:
print(X_train_full.shape)
print(Y_train_full.shape)
print(X_test.shape)


In [ ]:
X_train, x_val, Y_train, y_val = train_test_split(X_train_full, Y_train_full,
                                                  test_size=0.3, random_state=42)


In [ ]:
print(X_train.shape)
print(x_val.shape)
print(Y_train.shape)
print(y_val.shape)
print(X_test.shape)


In [ ]:
# plot random samples with keypoints from training
def plot_documents(nrows=3, ncols=2):
    selection = np.random.choice(len(X_train), size=(nrows*ncols), replace=False)
    images = np.asarray(train_images)[selection]
    cleaned_images = np.asarray(train_cleaned_images)[selection]
    #images = X_train[selection]
    #cleaned_images = Y_train[selection]
    fig, axes = plt.subplots(figsize=(nrows*20, ncols*30), nrows=nrows, ncols=ncols)
    fig.subplots_adjust(hspace = .05, wspace=.05)
    axes = axes.ravel()
    for img, img_cleaned, i in zip(images, cleaned_images, range(0, nrows*ncols, 2)):
        axes[i].imshow(cv2.imread(img, cv2.IMREAD_GRAYSCALE), cmap='gray')
        axes[i+1].imshow(cv2.imread(img_cleaned, cv2.IMREAD_GRAYSCALE), cmap='gray')
        axes[i].axis('off')
        axes[i+1].axis('off')
        
plot_documents()


In [ ]:
'''
# CNN model architecture (all these parameters might be tuned to achieve better results)
from functools import partial

DefaultConv2D = partial(Conv2D, activation='relu', padding='SAME')

model = Sequential([
    # encoder
    DefaultConv2D(filters=32, kernel_size=5, input_shape=[258,540,1]),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    DefaultConv2D(filters=64, kernel_size=3),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    # decoder
    DefaultConv2D(filters=64, kernel_size=3),
    BatchNormalization(),
    UpSampling2D((2, 2)),
    DefaultConv2D(filters=32, kernel_size=5),
    BatchNormalization(),
    UpSampling2D((2,2)),
    DefaultConv2D(filters=1, kernel_size=5, activation='sigmoid')
])

# show model architecture
model.summary()
'''


In [ ]:
N, H, W = X_train.shape
input_img = Input(shape=(H,W,1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

autoencoder.summary()


In [ ]:
# Lets' define our autoencoder now
def build_autoenocder():
    input_img = Input(shape=(258,540,1), name='image_input')
    
    #enoder 
    #x = ZeroPadding2D((1,0))(input_img)
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
    x = MaxPooling2D((2,2), padding='same', name='pool1')(x)
    #x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv2')(x)
    #x = MaxPooling2D((2,2), padding='same', name='pool2')(x)
    
    #decoder
    #x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv3')(x)
    #x = UpSampling2D((2,2), name='upsample1')(x)
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv4')(x)
    x = UpSampling2D((2,2), name='upsample2')(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv5')(x)
    #x = Cropping2D((1,0))(x)
    
    #model
    autoencoder = Model(inputs=input_img, outputs=x)
    return autoencoder

model = build_autoenocder()
model.summary()


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [ ]:
# another method is to use LearningRateScheduler, reduce the learning rate by 10% every epoch
# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
K = keras.callbacks
reduce_lr = K.ReduceLROnPlateau(monitor='val_mae', patience=7,
                                             verbose=1, factor=0.1, min_lr=0.00001)

early_stopping = K.EarlyStopping(monitor='val_mae', patience=20, restore_best_weights=True,
                                 verbose=1, mode='auto')
#checkpointer = K.ModelCheckpoint(filepath = 'best_model.hdf5', monitor='val_mae',
                                 #verbose=1, save_weights_only=True)


In [ ]:
# it is better to increase the batch size when the dataset is small
epochs = 300
batch_size = 8
history = model.fit(X_train, Y_train, validation_data=(x_val, y_val),
                   batch_size=batch_size, epochs=epochs, shuffle=True,
                   callbacks=[reduce_lr, early_stopping])


In [ ]:
final_loss, final_mae = model.evaluate(x_val, y_val, verbose=1)
print("Final loss: {0:.4f}, final mae: {1:.4f}".format(final_loss, final_mae))


In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['mae'], color='b', label="Training mae")
ax[1].plot(history.history['val_mae'], color='r',label="Validation mae")
legend = ax[1].legend(loc='best', shadow=True)


In [ ]:
# plot random samples with keypoints from training
def plot_documents_after_training(nrows=3, ncols=2):
    selection = np.random.choice(len(X_test), size=(nrows*ncols), replace=False)
    original_images = np.asarray(test_images)[selection]
    tested_images = X_test[selection]
    predicted_images = model.predict(tested_images)
    fig, axes = plt.subplots(figsize=(nrows*20, ncols*30), nrows=nrows, ncols=ncols)
    fig.subplots_adjust(hspace = .05, wspace=.05)
    axes = axes.ravel()
    for original_img, tested_img, predicted_img, i in zip(original_images, tested_images, predicted_images, range(0, nrows*ncols, 2)):
        original_img = cv2.imread(original_img, cv2.IMREAD_GRAYSCALE)
        axes[i].imshow(cv2.resize(tested_img, (original_img.shape[1],original_img.shape[0])), cmap='gray')
        axes[i+1].imshow(cv2.resize(predicted_img, (original_img.shape[1],original_img.shape[0])), cmap='gray')
        axes[i].axis('off')
        axes[i+1].axis('off')
        
plot_documents_after_training()


In [ ]:
# save the weights to prevent training every time you open the kernel
model.save_weights("model.h5")


In [ ]:
# after loading, you have to compile the model
model.load_weights('/kaggle/input/denoising-dirty-documents2/model.h5')
model.compile(optimizer='adam', loss='mse',
             metrics=['mae'])

final_loss, final_mae = model.evaluate(x_val, y_val, verbose=0)
print("Final loss: {0:.4f}, final mae: {1:.4f}".format(final_loss, final_mae))


In [ ]:
results = model.predict(X_test)
print(results.shape)


In [ ]:
def split_numbers(s):
    head = s.split('.')[0].split('/')[1]
    return head


In [ ]:
ids = []
values = []

for i, pred in enumerate(results):
    img_dir = test_images[i]
    img_id = split_numbers(img_dir)
    img = cv2.imread(img_dir,cv2.IMREAD_GRAYSCALE)
    pred = cv2.resize(pred, (img.shape[1], img.shape[0]))
    for j in range(pred.shape[1]):
        for k in range(pred.shape[0]):
            values.append(pred[k][j].item())
            ids.append(img_id + '_' + str(k+1) + '_' + str(j+1))
    print("Processed: {}".format(img_id))
    print(pred.shape)      


In [ ]:
len(ids)


In [ ]:
values


In [ ]:
pd.DataFrame({'id': ids, 'value': values}).to_csv('submission.csv', index=False)


In [ ]:
my_submission = pd.read_csv("submission.csv")


In [ ]:
my_submission.head()
